In [1]:
import pandas as pd
import numpy as np
import warnings

pd.set_option('display.max_rows', 500)
warnings.filterwarnings('ignore')

In [2]:
raw_data = """
E PASS
EG RW S 0
MB C L 0
MP RW S 0
E LE P 0
MP POST
E LE S 0
E P L 1
SN RC S 1
MP P L 1
E R L 0
E P L 0
MB TK S 1
E T S 0
MB C L 0
MB P L 0
MB P L 1
E LW S 0
SN P L 1
E RE P 0
GH LW S 1
E DP
E PASS
E P L 0
GH PER
E T S 0
AG LB S 1
E DP
IH RW S 0
E PASS
BA RC S 0
E R L 1
E P H 0
LC RE S 0
E P P 1
MP RE S 1
E PASS
IH P L 0
E TK P 0
EG LW S 0
E T B 0
EG P L 0
E P F 0
MP T S 0
E RB S 0
EG SCREEN
E PASS
MB R L 0
GH DRIVE
E L L 1
E P P 0
MB P L 1
E PASS
KG T S 0
E LB L 0
GH LW P 0
E R L 1
E P H 0
GH DP
E RW S 0
MB C S 1
MB P L 1
E P L 0
GH P L 1
E L L 0
E LW P 0
SN RW S 1
E P L 1
SN T S 0
GH RW S 0
BA PER
E RW S 0
BA T S 0
MB L L 1
MB P L 1
E L L 1
E LB H 0
SN RW B 0
SN P L 1
E L L 0
IH LC S 1
E P H 0
LC P L 1
E L L 1
E LB H 0
KG TK P 1
E L L 0
E P L 0
IH LC S 0
KG T S 0
IH P L 0
SN P L 1
E O L 0
E PASS
CW PASS
E P P 0
SN LW S 0
LC RE S 0
SN DP
E PASS
MP RE P 1
E L L 0
E P H 0
E DP
MB P L 1
E PASS
SN P L 0
GH T S 0
E R L 0
E RE H 0
EG RW S 1
E R L 1
E P H 0
E LW S 0
MB R L 0
MB P L 0
E P P 0
MB SCREEN
EG P L 1
E P H 0
EG LW S 0
E TK P 0
SN RC S 0
E L L 1
E LB H 0
GH P L 0
SN P L 1
E P L 0
SN T S 1
E P L 0
LC R L 0
BA DRIVE
E P L 0
LC LB S 0
AG LE S 0
BA T S 0
KG RC S 0
KG P L 1
E RW S 0
CW RW S 0
AG LB P 1
GH SCREEN
E PASS
GH P L 0
E P L 0
E P L 0
EG LW S 0
E POST
MB P L 1
E P L 1
MB TK S 1
E LW P 0
MB R L 0
MB C L 0
EG PASS
E T S 0
SN P L 1
E SCREEN
MP P L 1
E L L 1
EG T S 0
E P L 1
GH P L 1
E P L 1
E L S 1
MB C L 1
MB P H 0
E TK P 0
E P L 0
GH P L 1
E RW S 0
KG PER
E RE P 0
KG RW S 0
E P L 1
LC P L 1
LC P L 0
IH LC S 0
E P H 0
CW RB P 0
E LE P 1
KG RW S 0
CW LW S 1
E LW S 1
KG DRIVE
E P L 0
AG LB S 0
E P L 0
SN T S 0
LC POST
E P L 1
SN DRIVE
E TK S 1
BA T S 1
E C L 0
E RW P 0
KG PASS
E P L 0
KG P H 1
E DRIVE
LC P L 1
E SCREEN
KG PASS
E P L 1
KG RW S 0
E PASS
BA P L 1
E P L 0
IH PER
E P F 0
LC P L 1
LC R L 1
E PASS
MP P L 0
E C L 1
E LB H 0
MP PASS
E P L 1
IH LB P 0
CW LC S 0
E R L 0
E P L 0
SN PER
E P S 0
E PER
IH LW S 0
E RW S 0
IH RB L 0
IH P L 1
E T S 0
AG PASS
CW PASS
E P L 1
"""

In [3]:
rows = raw_data.split("\n")[1:-1]

raw_shots = "\n".join([i for i in rows if (i.split(" ")[1] not in ['L', 'C', 'R']) and (len(i.split(" ")) == 4)])
raw_posts = "\n".join([i for i in rows if i.split(" ")[1] in ['L', 'C', 'R']])
raw_tovs = "\n".join([i for i in rows if len(i.split(" ")) == 2])

## Abbreviations

In [4]:
players = {'ABR': 'Alec Bryan',
           'KO' : 'Kerem Ozturk',
           'LE' : 'Leonardo Edo',
           'ST' : 'Skyler Twyman',
           'EM' : 'Ezra Moos',
           'TK' : 'Thomas Kurowski',
           'DW' : 'Dashiel Walker',
           'EP' : 'Elliot Paschal',
           'AB' : 'Arrish Bhandal',
           'JP' : 'Josh Preston',
           'JX' : 'Jackson Piotrowski',
           'BH' : 'Bryce Hopkins',
           'TO' : 'Tola Olorode',
           'JB' : 'Joe Berry',
           'BC' : 'Ben Chasin',
           'BL' : 'Blake Hawthorne',
           'EK' : 'Eamonn Kenah',
           'XX' : 'Alex Battist',
           'EG' : 'Ellie Gross',
           'KG' : 'Kate Gross',
           'GH' : 'Grace Hynes',
           'SN' : 'Sophia North',
           'AG' : 'Ashley Gao',
           'IH' : 'Isabelle Herrera',
           'BA' : 'Bella Alfaro',
           'LC' : 'Lindsey Carter',
           'CW' : 'Caroline Workman',
           'MB' : 'Mallory Brodnik',
           'PV' : 'Peyton Van Soest',
           'AC' : 'Alexis Clark',
           'AW' : 'Amber Williams',
           'MP' : 'Marissa Powe',
           'E' : 'Edgewood'}

In [5]:
turnover_types = {'DRIVE' : 'Drive',
                  'DP' : 'Drive + Pass',
                  'PASS' : 'Reversal/Pass',
                  'PER' : 'Perimeter/Strip',
                  'POST' : 'Post Entry', 
                  'SCREEN' : 'Off-Ball',
                  'KEY' : 'Off-Ball'}

# Shots

In [6]:
regions3 = ['RC', 'LC', 'LW', 'RW', 'T']

In [7]:
shots = pd.DataFrame(raw_shots.split("\n"), columns=['Raw'])

shots['Player'] = shots.Raw.apply(lambda i : players[ i.split(" ")[0] ])
shots['Region'] = shots.Raw.apply(lambda i : i.split(" ")[1])
shots['Shot Type'] = shots.Raw.apply(lambda i : i.split(" ")[2])
shots['Outcome'] = shots.Raw.apply(lambda i : int( i.split(" ")[3] ))
shots.index = shots.Player
shots = shots[['Region', 'Shot Type', 'Outcome']]

shots['Range'] = shots.Region.apply(lambda i : i in regions3)

shots[['Region', 'Shot Type', 'Outcome']]

,Region,Shot Type,Outcome
Player,,,
Ellie Gross,RW,S,0
Marissa Powe,RW,S,0
Edgewood,LE,P,0
Edgewood,LE,S,0
Edgewood,P,L,1
Sophia North,RC,S,1
Marissa Powe,P,L,1
Edgewood,P,L,0
Mallory Brodnik,TK,S,1


# Post Entries

In [8]:
posts = pd.DataFrame(raw_posts.split("\n"), columns=['Raw'])

posts['Player'] = posts.Raw.apply(lambda i : players[ i.split(" ")[0] ])
posts['Region'] = posts.Raw.apply(lambda i : i.split(" ")[1])
posts['Distance'] = posts.Raw.apply(lambda i : i.split(" ")[2])
posts['Shot?'] = posts.Raw.apply(lambda i : int( i.split(" ")[3] ))
posts.index = posts.Player
posts = posts[['Region', 'Distance', 'Shot?']]

posts

,Region,Distance,Shot?
Player,,,
Mallory Brodnik,C,L,0
Edgewood,R,L,0
Mallory Brodnik,C,L,0
Edgewood,R,L,1
Mallory Brodnik,R,L,0
Edgewood,L,L,1
Edgewood,R,L,1
Mallory Brodnik,C,S,1
Edgewood,L,L,0


# Turnovers

In [9]:
if raw_tovs == "":
    tovs = pd.DataFrame(columns=['Player', 'Turnover Type'])
else:
    tovs = pd.DataFrame(raw_tovs.split("\n"), columns=['Raw'])

    tovs['Player'] = tovs.Raw.apply(lambda i : players[ i.split(" ")[0] ])
    tovs['Turnover Type'] = tovs.Raw.apply(lambda i : turnover_types[ i.split(" ")[1] ])
    tovs.index = tovs.Player
    tovs = tovs[['Turnover Type']]

tovs

,Turnover Type
Player,
Edgewood,Reversal/Pass
Marissa Powe,Post Entry
Edgewood,Drive + Pass
Edgewood,Reversal/Pass
Grace Hynes,Perimeter/Strip
Edgewood,Drive + Pass
Edgewood,Reversal/Pass
Edgewood,Reversal/Pass
Ellie Gross,Off-Ball


# Daily Totals

In [10]:
def filter_zero(values):
    for i in [2, 5, 8]:
        if values[i] == 0:
            values[i-1] = ""
            values[i] = ""
            values[i+1] = ""
            
    for i in [10, 11]:
        if values[i] == 0:
            values[i] = ""

    return values

In [11]:
fgs = shots['Outcome'].groupby('Player').agg([np.sum, len])
fgs.columns = ['FGM', 'FGA']
fgs['FG%'] = fgs['FGM'] / fgs['FGA'] 

twos = shots[(~shots.Range)]['Outcome'].groupby('Player').agg([np.sum, len])
twos.columns = ['2PM', '2PA']
twos['2P%'] = twos['2PM'] / twos['2PA']

threes = shots[(shots.Range)]['Outcome'].groupby('Player').agg([np.sum, len])
threes.columns = ['3PM', '3PA']
threes['3P%'] = threes['3PM'] / threes['3PA']

tov_totals = tovs.groupby('Player')['Turnover Type'].agg([len])
tov_totals.columns = ['Turnovers']

post_totals = posts.groupby('Player')['Shot?'].agg([len])
post_totals.columns = ['Post-Ups']

In [12]:
daily_total = pd.concat([fgs, twos, threes, tov_totals, post_totals], axis=1)
daily_total['Post-Ups'] = daily_total['Post-Ups'].fillna(0).astype(int)

daily_total = daily_total.fillna(0).sort_values(by=['FGA'], ascending=False)

daily_total['FG%'] = round(100 * daily_total['FG%'], 1)
daily_total['2P%'] = round(100 * daily_total['2P%'], 1)
daily_total['3P%'] = round(100 * daily_total['3P%'], 1)
daily_total['FGM'] = daily_total['FGM'].astype(int)
daily_total['FGA'] = daily_total['FGA'].astype(int)
daily_total['2PM'] = daily_total['2PM'].astype(int)
daily_total['2PA'] = daily_total['2PA'].astype(int)
daily_total['3PM'] = daily_total['3PM'].astype(int)
daily_total['3PA'] = daily_total['3PA'].astype(int)
daily_total['Turnovers'] = daily_total['Turnovers'].astype(int)

daily_total['PTS'] = (2*daily_total['2PM'] + 3*daily_total['3PM']).astype(int)
first_column = daily_total.pop('PTS')
daily_total.insert(0, 'PTS', first_column)

uc = daily_total[(daily_total.index != 'Edgewood')]
uc.loc['Totals'] = uc.sum()
uc = uc.astype(int)
uc['FG%'] = round(100 * uc['FGM'] / uc['FGA'], 1)
uc['2P%'] = round(100 * uc['2PM'] / uc['2PA'], 1)
uc['3P%'] = round(100 * uc['3PM'] / uc['3PA'], 1)

daily_total = pd.concat([daily_total, uc.tail(1)])

names_index = daily_total.index

daily_total = pd.DataFrame([filter_zero(list(i)) for i in daily_total.values],
                           columns=daily_total.columns)

daily_total.index = names_index

daily_total

,PTS,FGM,FGA,FG%,2PM,2PA,2P%,3PM,3PA,3P%,Turnovers,Post-Ups
Player,,,,,,,,,,,,
Edgewood,29.0,14.0,72.0,19.4,13.0,56.0,23.2,1.0,16.0,6.2,20.0,18.0
Sophia North,19.0,8.0,14.0,57.1,5.0,6.0,83.3,3.0,8.0,37.5,3.0,
Mallory Brodnik,16.0,8.0,11.0,72.7,8.0,11.0,72.7,,,,1.0,9.0
Isabelle Herrera,5.0,2.0,10.0,20.0,1.0,5.0,20.0,1.0,5.0,20.0,1.0,
Grace Hynes,9.0,4.0,9.0,44.4,3.0,5.0,60.0,1.0,4.0,25.0,4.0,
Kate Gross,6.0,3.0,9.0,33.3,3.0,3.0,100.0,0.0,6.0,0.0,4.0,
Ellie Gross,5.0,2.0,8.0,25.0,1.0,2.0,50.0,1.0,6.0,16.7,2.0,
Lindsey Carter,8.0,4.0,8.0,50.0,4.0,8.0,50.0,,,,1.0,2.0
Marissa Powe,8.0,4.0,7.0,57.1,4.0,5.0,80.0,0.0,2.0,0.0,2.0,


# Cumulative Totals

In [13]:
mbb_url = ["https://docs.google.com/spreadsheets/d/1BcIP7CIYDTNnedcRG3U3HAIwluaJtz8LCAKfArlEh98/edit#gid=2032963821".replace('/edit#gid=', '/export?format=csv&gid='),
            "https://docs.google.com/spreadsheets/d/1BcIP7CIYDTNnedcRG3U3HAIwluaJtz8LCAKfArlEh98/edit#gid=1904120862".replace('/edit#gid=', '/export?format=csv&gid='),
            "https://docs.google.com/spreadsheets/d/1BcIP7CIYDTNnedcRG3U3HAIwluaJtz8LCAKfArlEh98/edit#gid=2056191088".replace('/edit#gid=', '/export?format=csv&gid=')]

wbb_url = ["https://docs.google.com/spreadsheets/d/12JWqAMfVrSZobLohmxQK6PyrbNbOQbM6ux4LxXqWenM/edit#gid=2032963821".replace('/edit#gid=', '/export?format=csv&gid='),
           "https://docs.google.com/spreadsheets/d/12JWqAMfVrSZobLohmxQK6PyrbNbOQbM6ux4LxXqWenM/edit#gid=748348799".replace('/edit#gid=', '/export?format=csv&gid='),
           "https://docs.google.com/spreadsheets/d/12JWqAMfVrSZobLohmxQK6PyrbNbOQbM6ux4LxXqWenM/edit#gid=2056191088".replace('/edit#gid=', '/export?format=csv&gid=')]

In [14]:
mbb_game_dates = ["10/28/22", "11/1/22"]
wbb_game_dates = ["10/29/22", "11/5/22"]
teams_played = ['Hope', 'Elmhurst', 'Concordia', 'Edgewood']

In [15]:
current = wbb_url
current_dates = wbb_game_dates

shots_df = pd.read_csv(current[0])
posts_df = pd.read_csv(current[1])
tovs_df = pd.read_csv(current[2])

shots_df['Range'] = shots_df.Region.apply(lambda i : i in regions3)

shots_df = shots_df[(shots_df['Date'].isin(current_dates))]
posts_df = posts_df[(posts_df['Date'].isin(current_dates))]
tovs_df = tovs_df[(tovs_df['Date'].isin(current_dates))]

In [16]:
fgs = shots_df.groupby('Player')['Outcome'].agg([np.sum, len])
fgs.columns = ['FGM', 'FGA']
fgs['FG%'] = fgs['FGM'] / fgs['FGA']

twos = shots_df[(~shots_df.Range)].groupby('Player')['Outcome'].agg([np.sum, len])
twos.columns = ['2PM', '2PA']
twos['2P%'] = twos['2PM'] / twos['2PA']

threes = shots_df[(shots_df.Range)].groupby('Player')['Outcome'].agg([np.sum, len])
threes.columns = ['3PM', '3PA']
threes['3P%'] = threes['3PM'] / threes['3PA']

tov_totals = tovs_df.groupby('Player')['Turnover Type'].agg([len])
tov_totals.columns = ['Turnovers']

post_totals = posts_df.groupby('Player')['Shot?'].agg([len])
post_totals.columns = ['Post-Ups']

In [17]:
cumulative_total = pd.concat([fgs, twos, threes, tov_totals, post_totals], axis=1)
cumulative_total['Post-Ups'] = cumulative_total['Post-Ups'].fillna(0).astype(int)

cumulative_total = cumulative_total.fillna(0).sort_values(by=['FGA'], ascending=False)
cumulative_total = cumulative_total[~cumulative_total.index.isin(teams_played)]

cumulative_total['FG%'] = round(100 * cumulative_total['FG%'], 1)
cumulative_total['2P%'] = round(100 * cumulative_total['2P%'], 1)
cumulative_total['3P%'] = round(100 * cumulative_total['3P%'], 1)
cumulative_total['3PM'] = cumulative_total['3PM'].astype(int)
cumulative_total['3PA'] = cumulative_total['3PA'].astype(int)
cumulative_total['Turnovers'] = cumulative_total['Turnovers'].astype(int)

cumulative_total['PTS'] = (2*cumulative_total['2PM'] + 3*cumulative_total['3PM']).astype(int)
first_column = cumulative_total.pop('PTS')
cumulative_total.insert(0, 'PTS', first_column)

uc = cumulative_total[~(cumulative_total.index.isin(teams_played))]
uc.loc['Totals'] = uc.sum()
uc = uc.astype(int)
uc['FG%'] = round(100 * uc['FGM'] / uc['FGA'], 1)
uc['2P%'] = round(100 * uc['2PM'] / uc['2PA'], 1)
uc['3P%'] = round(100 * uc['3PM'] / uc['3PA'], 1)

cumulative_total = pd.concat([cumulative_total, uc.tail(1)])

names_index = cumulative_total.index

cumulative_total = pd.DataFrame([filter_zero(list(i)) for i in cumulative_total.values],
                                columns=cumulative_total.columns)

cumulative_total.index = names_index

cumulative_total

,PTS,FGM,FGA,FG%,2PM,2PA,2P%,3PM,3PA,3P%,Turnovers,Post-Ups
Player,,,,,,,,,,,,
Sophia North,29.0,13.0,25.0,52.0,10.0,11.0,90.9,3.0,14.0,21.4,5.0,
Mallory Brodnik,34.0,17.0,21.0,81.0,17.0,21.0,81.0,,,,2.0,15.0
Grace Hynes,16.0,7.0,18.0,38.9,5.0,12.0,41.7,2.0,6.0,33.3,5.0,
Isabelle Herrera,14.0,5.0,18.0,27.8,1.0,6.0,16.7,4.0,12.0,33.3,3.0,
Lindsey Carter,16.0,8.0,15.0,53.3,8.0,15.0,53.3,,,,1.0,5.0
Kate Gross,12.0,5.0,14.0,35.7,3.0,5.0,60.0,2.0,9.0,22.2,4.0,
Marissa Powe,17.0,8.0,14.0,57.1,7.0,10.0,70.0,1.0,4.0,25.0,5.0,1.0
Ellie Gross,5.0,2.0,12.0,16.7,1.0,4.0,25.0,1.0,8.0,12.5,3.0,
Caroline Workman,10.0,4.0,10.0,40.0,2.0,3.0,66.7,2.0,7.0,28.6,4.0,
